In [1]:

using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials
using Plots
using LaTeXStrings

In [132]:

# import conventions
include("conventions.jl")
  using .conventions: big_endian, qubit_begin

  # import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: Qgate_T2D

include("quantum_circuit.jl")
using ..quantum_circuit: qc_initialize, init_register, show_statevector, apply_op
#using ..quantum_circuit: qc_initialize, init_register, print_initstate

# import the sorting function
include("lib_useful/custom_functions.jl")
using .custom_functions: MK_sortrows


include("lib_useful/custom_functions.jl")
using ..custom_functions: MK_sortrows

Load quantum gates constructor


Load Tensor module: QTensor.jl
Load quantum gates constructor


Load quantum_circuit constructor
Load quantum gates constructor


Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum_circuit constructor


### quantum sensor test 

In [3]:
#qubit_control = 0
#qubit_target = 1
#nqubits = 1
err_tol = 1e-8
nqubits = 1
qc_le = qc_initialize(nqubits,"little-endian")

Main.quantum_circuit.qc_initstruct(1, "little-endian", 2, 2, [1.0, 0.0], [0, 1])

In [4]:
q_tab = [0;1] # initialize the basis vectors quantum table


2-element Vector{Int64}:
 0
 1

In [5]:
show_statevector(qc_le)

1.0 * | [0]>
0.0 * | [1]>


In [6]:
# global variable 
err_tol = 1e-8

1.0e-8

In [7]:

Base.@kwdef mutable struct qc_initstruct
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector::Union{Vector{Float64},Vector{Int64}, Vector{ComplexF64}}
#    state_vector::Array{Float64,1}
    q_states
#    q_states::Array{Float64,2}
end # end qc_initialize


qc_initstruct

In [ ]:

function qc_initialize2(n::Int64,
    q_order::String="big-endian",
    #big_endian::Bool=conventions.big_endian,
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol
    )
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end
        
   else
    n_count = n_bas
    if q_order == "big-endian"
       for i = 2:n_qubits
            q_tab = [zeros(n_count,1) q_tab
                    ones(n_count,1) q_tab]
            n_count = n_count*2
       end
    else
        for i = 2:n_qubits
            q_tab = [ q_tab zeros(n_count,1)
                   q_tab  ones(n_count,1) ]
            n_count = n_count*2
        end
    end
    q_states = q_tab # basis vectors 
end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

#    return 1

end # end qc_initialize  

In [125]:

function qc_initialize3(n::Int64,
    big_endian::Bool=conventions.big_endian,
    #big_endian::Bool=conventions.big_endian,
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol
    )
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   
    # store the order of the qubits, big-endian or little-endian
    # for printing purpose only 
    if big_endian
        q_order = "big-endian"
    else
        q_order = "little-endian"
    end

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end 
   else
        n_count = n_bas
        if !big_endian
            for i = 2:n_qubits
                q_tab = [ q_tab zeros(n_count,1)
                       q_tab  ones(n_count,1) ]
                n_count = n_count*2
            end
        else
            for i = 2:n_qubits
                q_tab = [zeros(n_count,1) q_tab
                        ones(n_count,1) q_tab]
                n_count = n_count*2
            end
        end
        # for i = 2:n_qubits
        #     q_tab = [zeros(n_count,1) q_tab
        #             ones(n_count,1) q_tab]
        #     n_count = n_count*2
        # end
    q_states = q_tab # basis vectors 
end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

#    return 1

end # end qc_initialize  

qc_initialize3 (generic function with 4 methods)

In [126]:
nqubits = 2
qc2 = qc_initialize3(nqubits,false)

qc_initstruct(2, "little-endian", 2, 4, [1.0, 0.0, 0.0, 0.0], [0.0 0.0; 1.0 0.0; 0.0 1.0; 1.0 1.0])

In [127]:
qc2

qc_initstruct(2, "little-endian", 2, 4, [1.0, 0.0, 0.0, 0.0], [0.0 0.0; 1.0 0.0; 0.0 1.0; 1.0 1.0])

In [128]:
qc2.state_vector



4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [129]:
qc2.q_order

"little-endian"

In [115]:

# print the initial state of the quantum register
# print the initial state of the quantum register
function show_statevector_test(qc)
    # print the initial state of the quantum register
    # qc::qc_initstruct: quantum register
    # return: print the initial state of the quantum register
    #println("The initial state of the quantum register is: ")
    #println(qc.state_vector)
    # print the initial state of the quantum register with the quantum 
    # states in the computational basis
    #println("the quantum register is: ")
    q_states = qc.q_states
    state_vector = qc.state_vector
    if qc.q_order == "little-endian"
        q_states, q_ind = MK_sortrows22(q_states)
        state_vector = state_vector[q_ind[:,1]]
    end # end if
    
    # Note: the basis of the Hilbert space are the quantum states
    # .. they are arranged and sorted in the computational basis 
    # .. according to the q_order=lille-endian.
    # .. accordingly, the statevector is arranged and sorted in the
    # .. computational basis according to the q_order=little-endian 
    # .. as well. 

    # convert the quantum states to integers
    q_table = zeros(Int, qc.n_dim, qc.n_qubits)
    for iq=1:qc.n_qubits
        for i in 1:qc.n_dim
            q_table[i,iq] = trunc(Int, q_states[i,iq])
        end # end for
    end # end for
  
    for i in 1:qc.n_dim
        println( state_vector[i], " * | ", string(q_table[i,:]), ">")
    end # end for
    #show(stdout, "text/plain", [qc.state_vector trunc(Int,qc.q_states)])
end # end print_initstate

show_statevector_test (generic function with 1 method)

In [123]:
show_statevector_test(qc2)

1.0 * | [0, 0]>
0.0 * | [0, 1]>
0.0 * | [1, 0]>
0.0 * | [1, 1]>


In [106]:
# write the function MK_sortrows22 with an input matrix of union types of Float64 and Int64
function MK_sortrows22(A::Matrix{Float64})
    # sort the rows of the matrix A and export the sorting indices
    # A::Array{Float64,2}: matrix to be sorted
    # return: sorted matrix and sorting indices
    # more references over the sorting 
    # https://docs.julialang.org/en/v1/base/sort/
    # sort the rows of the matrix qc_le.q_states and export the sorting indices
    A_sorted=sortslices(A,dims=1)
    # where are the indices of the sorting?
    A_sorted_indices=sortperm(A,dims=1)
    return A_sorted, A_sorted_indices
end # end sortrows2


MK_sortrows33 (generic function with 1 method)

In [107]:
M_val,M_ind = MK_sortrows22(qc2.q_states)

([0.0 0.0; 0.0 1.0; 1.0 0.0; 1.0 1.0], [1 5; 3 6; 2 7; 4 8])

In [111]:
# print the first column of the sorted matrix
M_ind[:,1]

4-element Vector{Int64}:
 1
 3
 2
 4

In [ ]:
M_val

In [ ]:
M_val

In [ ]:
function show_statevector2(qc)
    # print the initial state of the quantum register
    # qc::qc_initstruct: quantum register
    # return: print the initial state of the quantum register
    #println("The initial state of the quantum register is: ")
    #println(qc.state_vector)
    # print the initial state of the quantum register with the quantum 
    # states in the computational basis
    #println("the quantum register is: ")
    q_table = zeros(Int, qc.n_dim, qc.n_qubits)
    for iq=1:qc.n_qubits
        for i in 1:qc.n_dim
            q_table[i,iq] = trunc(Int, qc.q_states[i,iq])
        end # end for
    end # end for

    for i in 1:qc.n_dim
        println( qc.state_vector[i], " * | ", string(q_table[i,:]), ">")
    end # end for
    #show(stdout, "text/plain", [qc.state_vector trunc(Int,qc.q_states)])
end # end print_initstate

In [ ]:
show_statevector2(qc_le)

In [ ]:
    n = 2
#    q_order = "big-endian"
    q_order = "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end
        
   else
    n_count = n_bas
    if q_order == "big-endian"
       for i = 2:n_qubits
            q_tab = [zeros(n_count,1) q_tab
                    ones(n_count,1) q_tab]
            n_count = n_count*2
       end
    elseif q_order == "little-endian"
        for i = 2:n_qubits
            q_tab = [ q_tab zeros(n_count,1)
                   q_tab  ones(n_count,1) ]
            n_count = n_count*2
        end
    end
    q_states = q_tab # basis vectors 

end # end if n_qubits == 1

#### testing the rotational gate
$R_Z(\phi)$.

syntax for the gate is: `Rz_gate = Qgate.Rz(phi)`

where `phi` is the angle of rotation.

In [ ]:
# a gate from the Qgate library implemented from the paper 
# arXiv:2209.08187v1 [quant-ph] 16 Sep 2022
# test angle 2.0
Rz_gate = Rz_gate1(2.0)


In [ ]:
show(stdout, "text/plain", Rz_gate)

### ===============================

Now, we start with the implementation of the quantum circuit simulating the quantum sensor.

In [ ]:
qubit_control = 0
qubit_target = 1
# Start with one qubit only and one classical bit to recieve the answer
nqubits = 1
#err_tol = 1e-15
qc = qc_initialize(nqubits)

In [ ]:
# check the state vector of the quantum register
qc.state_vector

In [ ]:
# check the quantum states, the basis of the Hilbert space of the qubits
qc.q_states

In [ ]:
# print the state vector with the basis of the quantum register
show_statevector(qc)

In [ ]:
# Get the Hadamard gate
 Hgate = Qgate.H
# Hgate'*Hgate   
# Qgate_dim = size(Hgate)
# U2_mat::Matrix{Union{ComplexF64, Float64}} = Hgate'*Hgate
# #UU::Matrix{Union} = Hgate'*Hgate
# err_tol = 1e-15
# II = Matrix(I, Qgate_dim[1], Qgate_dim[2])
#     if isapprox(U2_mat, II,rtol=err_tol) == false
#         error("The gate is not unitary")
#     end

In [ ]:
# Apply the Hadamard gate to the quantum register
apply_op(qc, Hgate)

In [ ]:
# Show the state vector of the quantum register
show_statevector(qc)

In [ ]:
# Apply the Rz gate as defined in the paper arXiv:2209.08187v1 [quant-ph] 16 Sep 2022
# a gate from the Qgate library implemented from the paper 
# arXiv:2209.08187v1 [quant-ph] 16 Sep 2022
# angle is phi = pi/3
Rz_gate = Rz_gate1(pi/3)
apply_op(qc, Rz_gate)

In [ ]:
# Show the state vector of the quantum register
show_statevector(qc)

In [ ]:
# Apply the Hadamard gate to the quantum register
apply_op(qc, Hgate)

In [ ]:
show_statevector(qc)

The final state is expressed as 
$$
|\psi\rangle=H R_Z(\phi) H|0\rangle
$$

In matrix notation: 
$$
|\psi\rangle=\frac{1}{2}\left[\begin{array}{l}
e^{-i \phi / 2}+e^{i \phi / 2} \\
e^{-i \phi / 2}-e^{i \phi / 2}
\end{array}\right]=\left[\begin{array}{c}
\cos (\phi / 2) \\
-i \sin (\phi / 2)
\end{array}\right]
$$

Here, the probabilities of measuring the final state of the circuit as $|0\rangle$ and $|1\rangle$ are given by $p_0=\cos ^2(\phi / 2)$ and $p_1=\sin ^2(\phi / 2)$, respectively.

We summarize the result of this measurement by defining the polarization operator $\hat{P}$ such that the polarization is equal to -1 if we measure $|0\rangle$ and equal to 1 if we measure $|1\rangle$. Up to the overall sign, this is the $z$ coordinate of the Bloch vector. In matrix form, the polarization operator is
$$
\hat{P}=\left[\begin{array}{cc}
-1 & 0 \\
0 & 1
\end{array}\right] .
$$
For the final state $|\psi\rangle$ from Eq. (7), the expectation value of the polarization operator is
$$
\langle\psi|\hat{P}| \psi\rangle=p_1-p_0=\frac{1}{2} \sin ^2(\phi / 2)-\frac{1}{2} \cos ^2(\phi / 2)=-\cos \phi
$$

The coefficients of state $|\psi\rangle$ expressed in the computational basis of a single qubit gives the probability of measuring the state $|0\rangle$ or $|1\rangle$.
The average polarization is given by 
$$
P = p_1 - p_0 
$$ 
with each measurement outcome.  

In [ ]:
phi_vec = [0,pi/2,pi,3*pi/2,2*pi]

Use the exact simulator approach, we can calculate the average polarization for a given $\phi$.

In [ ]:
# calculate the polarization P for the phi_vec 
P = zeros(length(phi_vec))
for iphi = 1:length(phi_vec)
    phi = phi_vec[iphi]
    p0 = cos(phi/2)^2
    p1 = sin(phi/2)^2
    P[iphi] = p0 - p1
end

In [ ]:
# plot P as a function of phi 
plot(phi_vec, P, label="Polarization", xlabel="phi", 
ylabel="P", title="Polarization as a function of phi")

# Apply the Rz gate as defined in the paper arXiv:2209.08187v1 [quant-ph] 16 Sep 2022

In [ ]:
## Emulate the sinusoidal behavior of the polarization P as a function of phi
P_polarization 

#### refs

https://quantumcomputing.stackexchange.com/questions/14066/how-do-i-apply-the-hadamard-gate-to-one-qubit-in-a-two-qubit-pure-state
